# Homework 4: Extending the GLM to include the following:
1. Calculate t-tests for each $beta$
2. Calculate an F-statistic, df, and P-value for the overall model $R^2$, comparing the full model to an intercept-only model. 
3. Add the ability to perform t-tests for a set of contrasts.
4. Make your script into a function that takes in a design matrix $X$, data $y$, and optional contrast matrix $C$. 

Pulling in functions from last assignment

In [59]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [45]:
# estimate the beta coefficients
def get_beta_estimates(X, y):
    betas = np.linalg.inv(X.T @ X) @ np.dot(X.T, y)
    return betas

In [46]:
# estimate the covariance of beta hat
def cov_beta_hat(X, y, B):
    S = (y - (X @ B)).T @ (y - (X @ B))
    n = X.shape[0]
    return S/(n-1)

In [47]:
# estimate the sum of squared errors
def SS_tot(y):
    return np.dot((y - np.mean(y)).T, (y - np.mean(y)))

In [48]:
# estimate the sum of squared residuals
def SS_res(X, y, B):
    e = y - np.dot(X, B)
    return np.dot(e.T, e)
# divide this output by n-p to get the variance of the residuals

In [49]:
# estimate the R squared
def R_squared(X, y, B):
    return (SS_tot(y) - SS_res(X, y, B))/SS_tot(y)

In [50]:
# put it all together
def GLM(X, y):
    B = get_beta_estimates(X, y)
    S = SS_res(X, y, B)
    n = X.shape[0]
    cov_beta = cov_beta_hat(X, y, B)
    R2 = R_squared(X, y, B)
    return B, S, cov_beta, R2

In [68]:
backpain = pd.read_csv('../week2/backpain.csv')
# subset to variables of interest
var_labels = ['id', 'group', 'bpi_intensity', 'gender', 'is_patient']
X = backpain[var_labels].to_numpy()
y = backpain['pain_diff'].to_numpy()
data.head()

,Unnamed: 0,id,1,2,group,bpi_intensity,gender,is_patient,pain_diff
0,0,12,2.50,2.75,3.0,2.50,1.0,1,0.25
1,2,14,2.50,2.00,3.0,2.50,2.0,1,-0.50
2,4,15,2.25,2.75,3.0,2.25,1.0,1,0.50
3,6,18,4.50,2.25,2.0,4.50,1.0,1,-2.25
4,8,23,2.50,2.25,2.0,2.50,2.0,1,-0.25


# 1. Calculate t-tests for each $\beta$

The t-statistic for testing $\beta$ = 0 is given by:
$t_j = \frac{\hat{\beta}_j}{\hat{\sigma}\sqrt{(X^T X)^{-1}_{jj}}}$

Within that function, I see that I need to calculate the following:
1. $\hat{\beta}_j$ - the estimate of the beta coefficient for the jth predictor
2. $\hat{\sigma}$ - the estimated standard deviation of the residuals
3. $(X^T X)^{-1}_{jj}$ - the jth diagonal element of the inverse of the product of the design matrix and its transpose. This essentially means I am calculating the covariance matrix of the design matrix, inverting it, and then taking the jth diagonal element of that matrix (where j is the index of the predictor in the design matrix).
        - Inside this function term, I see $X^T X$. This is the product of the design matrix and its transpose, which I know to be the covariance matrix of the design matrix. 

My understanding is that the t-statistic is the ratio of the estimated coefficient to the estimated standard deviation of the residuals. The estimated standard deviation of the residuals is the square root of the sum of squared residuals divided by the degrees of freedom. The degrees of freedom is the number of observations minus the number of predictors.

So, reducing the the standard deviation of the residuals would mean that I would see less spread of the data points around the regression line. Less spread means that my beta can be smaller but still be significant. In the same vein, if I have a large spread around that line, then my beta would have to be larger to be significant.

Also, adding more data points to an otherwise very noisy cloud of data points surrounding the regression line would reduce the standard deviation of the residuals. This is because the sum of squared residuals would be larger, but the degrees of freedom would also be larger. So, the ratio of the two would be smaller.

In [69]:
# make a function for the estimated variance of the residuals
def sigma_hat(X, y, B):
    n = X.shape[0]
    p = X.shape[1]
    return SS_res(X, y, B)/(n-p)

def t_stat(X, y, B):
    n = X.shape[0] # number of observations
    p = X.shape[1] # number of predictors
    # get sigma_hat (estimated variance of residuals)
    t = {}
    for j, varname in enumerate(var_labels):
        t[varname] = B[j] / (sigma_hat(X,y,B)**2 * np.sqrt(np.linalg.inv(X.T @ X)[j,j]))
        # reminder - try replacing the * with @ to see if that changes anything.
        
    return t
        
    """
    This function is an algorithm. For each beta, it finds the index of the beta in the beta vector. This is provided 
    by the j term of the enumerate function in the loop. The name of the variable, `varname`, is also provided as the second
    value of each iteration of the loop. The variable `t` is a dictionary. The key is the variable name, and the value is the
    t-statistic for that variable (these key-value pairs are populated on each successive iteration of the loop. 
    
    On each iteration, the t-statistic is calculated by dividing the beta estimate Bj by the product of the estimated standard 
    deviation of the residuals. 
    
    The function returns the dictionary of t-statistics.
    """

Below I'm gonna first get the betas and other values from the GLM function I wrote in the last assignment. Then I'm gonna use those values to calculate the t-statistic for each beta.

In [70]:
B, S, cov_beta, R2 = GLM(X, y)
for i, var in enumerate(var_labels):
    print(f'{var} beta = : {B[i]}')
print('\nSum of squared residuals: ', S)
print('Covariance of beta hat: ', cov_beta)
print('R squared: ', R2)

id beta = : 0.0003140003130136992
group beta = : 1.0560804398642403
bpi_intensity beta = : 0.009906787275012041
gender beta = : -0.1855060030488982
is_patient beta = : -3.429480968417778

Sum of squared residuals:  215.02230361367157
Covariance of beta hat:  1.6046440568184446
R squared:  0.33238629060675456


Now I'm gonna use the t_stat function I wrote above to calculate the t-statistic for each beta.

In [71]:
tvals = t_stat(X, y, B)
print('T-statistics: ', tvals)

T-statistics:  {'id': 0.5445476669625823, 'group': 3.624534678997911, 'bpi_intensity': 0.06343298420211381, 'gender': -0.3926125821888913, 'is_patient': -3.00887657599113}


Yay! Now, for sanity's sake, I'm gonna compare my t-statistics to the ones from the statsmodels package.

In [72]:
X_const = sm.add_constant(X)
model = sm.OLS(y, X_const)
results = model.fit()
print(results.summary()) 
print(results.tvalues)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.332
Model:                            OLS   Adj. R-squared:                  0.312
Method:                 Least Squares   F-statistic:                     16.18
Date:                Wed, 27 Sep 2023   Prob (F-statistic):           8.88e-11
Time:                        16:14:23   Log-Likelihood:                -222.98
No. Observations:                 135   AIC:                             456.0
Df Residuals:                     130   BIC:                             470.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0003      0.000      1.158      0.2

Whoops, looks like I made a mistake. My hand-calculated t-values are much smaller than the ones from the statsmodels package. However, they are all smaller by the same proportion. So I'm close, but clearly something is wrong with how I calculated them. 

Again, here's the formula for the t-statistic:
$t_j = \frac{\hat{\beta}_j}{\hat{\sigma}\sqrt{(X^T X)^{-1}_{jj}}}$

And here's my code:
```
def t_stat(X, y, B):
    n = X.shape[0] # number of observations
    p = X.shape[1] # number of predictors
    # get sigma_hat (estimated variance of residuals)
    t = {}
    for j, varname in enumerate(var_labels):
        t[varname] = B[j] / (sigma_hat(X,y,B)**2 * np.sqrt(np.linalg.inv(X.T @ X)[j,j]))
        # reminder - try replacing the * with @ to see if that changes anything.
        
    return t
```

The problem is that I'm squaring the estimated standard deviation of the residuals in `sigma_hat(X,y,B)**2`, where I should be taking the square root! This is because my function for $\hat\sigma$ actually returns the sum of squared residuals divided by the degrees of freedom. So, I need to take the square root of that value to get the estimated standard deviation of the residuals (perhaps I need to rename the function). 

In [84]:
def t_stat(X, y, B):
    n = X.shape[0] # number of observations
    p = X.shape[1] # number of predictors
    # get sigma_hat (estimated variance of residuals)
    t = {}
    for j, varname in enumerate(var_labels):
        ### NOW TAKE SQRT ###
        t[varname] = B[j] / (np.sqrt(sigma_hat(X,y,B)) * np.sqrt(np.linalg.inv(X.T @ X)[j,j]))
        
    return t

In [85]:
tvals = t_stat(X, y, B)
print('T-statistics: ', tvals)

T-statistics:  {'id': 1.1583668987127718, 'group': 7.710144125318964, 'bpi_intensity': 0.13493523826142437, 'gender': -0.8351691629908486, 'is_patient': -6.40051044086071}


In [86]:
# saving these to put in the table later
tstats = list(tvals.values())

Great - now I have the right t-values. Next I'm going to calculate the p-values for each t-statistic.

From the book:
The P-value comes from a t-distribution, which is a Normal (Gaussian) distribution adjusted for the fact that there are not really $n$ independent observations, and thus not really $n$ error df, if we have estimated some parameters ($p$ parameters, to be precise) and removed them when estimating the residual error. The t-statistic, $t_j$, can be compared to a t-distribution with $n - p$ degrees of freedom to determine the significance of the $j$-th predictor.

So to get p-values I really need to generate a null <em>distribution</em>, rather than a singular value. I think the best way to do this in Python is to use pre-made functionality to generate a cumulative distribution function (CDF) for the t-distribution, and then use that to calculate the p-value for each t-statistic. I'll use the `stats` module within the scipy package, and the `t.sf` function to calculate the p-value for each t-statistic. I'll also assume we want a two-tailed distribution for now, so I'll multiply the p-value by 2.

The `t.sf` function takes two arguments: the t-statistic and the degrees of freedom. It represents the survival function, which is the complement of the cumulative distribution function (CDF). 

In [87]:
from scipy import stats

In [88]:
# get the degrees of freedom
n = X.shape[0]
p = X.shape[1]
df = n - p

In [90]:
# get the p-values
pvals = {}
for varname, tstat in tvals.items():
    pvals[varname] = stats.t.sf(np.abs(tstat), df)*2
    
# saving these for later
pvalues = list(pvals.values())
print('P-values: ', pvals)

P-values:  {'id': 0.24883791360348023, 'group': 2.8747951957998716e-12, 'bpi_intensity': 0.8928717759108604, 'gender': 0.40515472362610394, 'is_patient': 2.5797920691951886e-09}


Now I'll integrate all this into a new and improved GLM function.

In [93]:
def fancy_GLM(X, y, predictor_names):
    B, S, cov_beta, R2 = GLM(X, y)
    tvals = t_stat(X, y, B)
    pvals = {}
    for varname, tstat in tvals.items():
        pvals[varname] = stats.t.sf(np.abs(tstat), df)*2
        
    output = pd.DataFrame({'Predictor': predictor_names, 'Beta': B, 'T-statistic': tstats, 'P-value (two-tailed)': pvalues})
    return output

In [94]:
new_model_results = fancy_GLM(X, y, var_labels)
new_model_results

,Predictor,Beta,T-statistic,P-value (two-tailed)
0,id,0.000314,1.158367,2.488379e-01
1,group,1.056080,7.710144,2.874795e-12
2,bpi_intensity,0.009907,0.134935,8.928718e-01
3,gender,-0.185506,-0.835169,4.051547e-01
4,is_patient,-3.429481,-6.400510,2.579792e-09


# 2. Calculate an F-statistic, df, and P-value for the overall model $R^2$, comparing the full model to an intercept-only model.

From the book:

The F-test is used to compare the fits of different models. Specifically, it is used to test the hypothesis that a set of predictors has no effect on the response variable. 

Given:

 $SSR_full$ = Sum of Squared Residuals for the full model

 $SSR_reduced$ = Sum of Squared Residuals for a reduced model

 $p$ = Number of predictors in the full model

 $q$ = Number of predictors in the reduced model (with)

 $n$ = Total number of observations

The F-statistic is caluclated as:


$$F = \frac{(SSR_{\text{reduced}} - SSR_{\text{full}}) / (p - q)}{SSR_{\text{full}} / (n - p)}$$

The difference between $SSR_reduced$ and $SSR_full$ is that $SSR_reduced$ is the sum of squared residuals for the reduced model, which is the model with fewer predictors. $SSR_full$ is the sum of squared residuals for the full model, which is the model with more predictors.


In [105]:
# calculate the sum of squared residuals for the intercept-only model (this just means like... the mean of the outcome variable)
X_intercept = np.ones((X.shape[0], 1)) # this is a column of ones - statsmodels can do this with sm.add_constant(X)
B_intercept, S_intercept, cov_beta_intercept, R2_intercept = GLM(X_intercept, y) 
print('Sum of squared residuals for intercept-only model: ', S_intercept) # this is SSreduced

Sum of squared residuals for intercept-only model:  322.07592592592596


In [106]:
# calculate the explained variance
explained_variance = S_intercept - S
print('Explained variance: ', explained_variance) # this is SSRreduced - SSRfull

Explained variance:  107.05362231225439


In [108]:
def get_F(X, y, B):
    n = X.shape[0] # number of observations
    p = X.shape[1] # number of predictors in the full model
    q = 1 # number of predictors in the reduced model (only 1 here bc intercept-only model)
    SSR_full = SS_res(X, y, B) # ss for full model
    SSR_reduced = SS_res(np.ones((n, 1)), y, B) # ss for intercept-only model (reduce model)
    F = ((SSR_reduced - SSR_full)/(p-1))/(SSR_full/(n-p))
    return F

F = get_F(X, y, B)
print('F-statistic: ', F)

ValueError: shapes (135,1) and (5,) not aligned: 1 (dim 1) != 5 (dim 0)

To get the p-value for the F I'm using `scipy.stats.f.sf`, which is like the function I used for the t-statistic, but for the F-statistic. It takes three arguments: the F-statistic, the degrees of freedom for the numerator, and the degrees of freedom for the denominator. The degrees of freedom for the numerator is the number of predictors in the full model minus the number of predictors in the intercept-only model (in our case, . The degrees of freedom for the denominator is the number of observations minus the number of predictors in the full model.

In [102]:
# get the p-value for the F-statistic
pval_F = stats.f.sf(F, 1, 1)